In [ ]:
def findDynGenes(expDat, sampTab, group_column="leiden", pseudotime_column="dpt_pseudotime"):


    sampTab["dpt_groups"]=sampTab[group_column]
    sampTab["psuedotime"]=sampTab[pseudotime_column]
    sampTab["cell_name"]=sampTab.index
    path=np.unique(sampTab["leiden"])
    ids=[]
    for grp in path:
        a=sampTab.loc[sampTab["dpt_groups"]==grp]
        b=a["cell_name"]
        ids=np.append(ids,b)
    sampTab=sampTab.loc[ids,:]
    expDat=expDat[ids]
    t1=sampTab["psuedotime"]
    t1C=t1[ids]
    print("starting gamma...")

    gpChr=gamFit(expMat=expDat[t1C.index],genes=expDat.index,celltime=t1C)

    cells=pd.DataFrame()
    cells["cell_name"]=pd.DataFrame(t1).index
    cells["psuedotime"]=t1.values
    cells["group"]=sampTab["dpt_groups"].values
    cells.index=cells["cell_name"]
    cells=cells.sort_values(by="psuedotime")
    ans=list([gpChr,cells])
    return ans

In [ ]:
def reconstructGRN(exp,zThresh):
    texp=exp.T #transpose expression matrix
    mim=pd.DataFrame()#create empty mim dataframe
    #create dataframe with mutual infrormation for each gene pair 
    #texp is the expression matrix with the genes as columns and the samples as rows
    for i in texp.columns:
        throwaway=[]
        for j in texp.columns:
            a=texp[i] 
            b=texp[j]
            mi_pair=normalized_mutual_info_score(a,b)
            throwaway.append(mi_pair)
        mim[i]=throwaway
    mim=mim.set_index(texp.columns)
    mim=mim.replace(1, 0)  # for replacing 1 to 0
    #calculate mean and std MI for each gene
    means=np.mean(mim,axis=1)
    stds=np.std(mim,axis=1)
    #calculate z scores for each gene using this formula: zi=max(0, (I(Xi;Xj)-mui)/sigmai)
    z_scores=[]
    genes=mim.columns
    for i in np.arange(len(genes)):
        column=genes[i]
        maximum=max(np.array(((mim[column]-means[i])/stds[i]).tolist()))
        z1=max(0,maximum)
        z_scores.append(z1)
    #calculate z scores from mutual information using formula: zij=sqrt(zi^2 + zj^2)
    xnet=pd.DataFrame()
    for i in np.arange(len(genes)):
        throwaway=[]
        for j in np.arange(len(genes)):
            zij=np.sqrt(z_scores[i]**2+z_scores[j]**2)
            throwaway.append(zij)
        xnet[mim.columns[i]]=throwaway
    xnet=xnet.set_index(mim.columns)
    #create correlation matrix
    xcorr=texp.corr()
    #transcription factors names
    tf=xcorr.columns
    grn=cn_extractRegsDF(xnet,xcorr,zThresh)
    return grn

In [ ]:
def cn_extractRegsDF(xnet, xcorr,zThresh):
    targets=[None]*10000000
    regulators=[None]*10000000
    zscoresX=[0]*10000000
    correlations=[0]*10000000
    start=0
    stp=0
    genes=xnet.columns
    for i in genes:
        x=pd.DataFrame(xnet[i])
        regs=list(x.loc[(x[i]>zThresh)].index)#zthresh
        if len(regs)>0:
            zzs=x.loc[regs]
            zzs=zzs[i].to_list()
            corrs=xcorr.loc[regs,i].to_list()
            ncount=len(regs)
            stp=start+ncount
            for j in np.arange(start,stp):
                targets[j]=i
            for j in np.arange(len(regs)):
                regulators[start+j]=regs[j]
            for j in np.arange(len(zzs)):
                zscoresX[start+j]=zzs[j]
            for j in np.arange(len(corrs)):
                correlations[start+j]=corrs[j]
            start=stp
    targets=targets[0:stp]
    regulators=regulators[0:stp]
    zscoresX=zscoresX[0:stp]
    correlations=correlations[0:stp]
    grn=pd.DataFrame()
    grn["TG"]=targets
    grn["TF"]=regulators
    grn["Zscores"]=zscoresX
    grn["Correlation"]=correlations
    return grn